# 1. 크롤링

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time
import pandas as pd
import re
import pickle

In [8]:
service = Service('C:/Users/khb16/Desktop/code/PROJECT/Recommandation/chromedriver.exe')
driver = webdriver.Chrome(service = service)

In [ ]:
pages = [] # 크롤링 오류 확인 위해 page 수 기록
title = []
lyric = []
artist = []
genres = []
writer = []
rank = []
p = re.compile("'(\d+)'")
    
for page in range(11401,15001,50): # 1페이지만 추출(1,51,50), 발라드 gnrCode=GN0100, 댄스 gnrCode=GN0200
    url = "https://www.melon.com/genre/song_list.htm?gnrCode=GN0200#params%5BgnrCode%5D=GN0200&params%5BdtlGnrCode%5D\
                =&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex={}".format(page) # 댄스 페이지
    driver.get(url)
    time.sleep(1)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 한 페이지에 있는 50개의 songId 추출
    songid = [] # songid

    for i in range(50):
        data = soup.find_all("a", class_= "btn button_icons type03 song_info")[i].get('href')
        id_ = p.search(str(data)).group(1)
        songid.append(id_)
        f = soup.select('td span.rank')[i].text
        rank.append(f)

    # 해당 songId 입력 후 정보 추출
    for song in songid:
        lyric_url = 'https://www.melon.com/song/detail.htm?songId={}'.format(song)
        driver.get(lyric_url)
        time.sleep(1)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        a = soup.select('div.song_name')[0].text[3:].strip() # 제목
        title.append(a)
        try:
            driver.find_element_by_css_selector(".button_more.arrow_d").click() # 가사
            ly = driver.find_element_by_css_selector('.lyric.on')
            b = ly.text
            lyric.append(b)
        except:
            lyric.append('가사없음')
        ar = driver.find_element_by_css_selector(".artist")
        c = ar.text
        artist.append(c)
        d = soup.select('dl.list dd')[2].text # 장르
        genres.append(d)
        try:
            e = soup.select('div.section_prdcr a.artist_name')[0].text # 작사가 1명
            writer.append(e)
        except:
            writer.append('작사가없음')
        pages.append(page)

<ipython-input-65-612ccfb23e46>:39: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector(".button_more.arrow_d").click() # 가사
<ipython-input-65-612ccfb23e46>:40: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  ly = driver.find_element_by_css_selector('.lyric.on')
<ipython-input-65-612ccfb23e46>:45: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  ar = driver.find_element_by_css_selector(".artist")


In [55]:
col = ['pages','rank','title','lyric','artist','genres','writer']
songlist = pd.DataFrame([pages, rank,title, lyric, artist, genres, writer]).T
songlist.columns = col
songlist

,pages,rank,title,lyric,artist,genres,writer
0,10901,10901,Never Thought (I'd Fall In Love),Never thought I’d fall in love\nNever thought\...,더 이스트라이트 (TheEastLight.),댄스,김희덕
1,10901,10902,TEENAGER (틴에이저) (Feat. 이로한),숨이 꽉 막혀\n이젠 눈물을 닦아\n캄캄한 어둠에 갇혀 복받쳐\n이것 좀 해라 저것...,사무엘,댄스,용감한 형제
2,10901,10903,Take my hand,That's when I love you\nLove ya love ya love y...,틴탑,댄스,zomay
3,10901,10904,HOTLINE,Boom I like it I like it\nBoom I like it I lik...,승리,댄스,승리
4,10901,10905,Enough,나는 알 수 있어\nWhat makes you sad\n언제든 지칠 때 기대 언제든...,GOT7 (갓세븐),댄스,JAY B
...,...,...,...,...,...,...,...
545,None,11446,None,None,None,None,None
546,None,11447,None,None,None,None,None
547,None,11448,None,None,None,None,None
548,None,11449,None,None,None,None,None


In [60]:
songlist = songlist.head(500)
songlist

,pages,rank,title,lyric,artist,genres,writer
0,10901,10901,Never Thought (I'd Fall In Love),Never thought I’d fall in love\nNever thought\...,더 이스트라이트 (TheEastLight.),댄스,김희덕
1,10901,10902,TEENAGER (틴에이저) (Feat. 이로한),숨이 꽉 막혀\n이젠 눈물을 닦아\n캄캄한 어둠에 갇혀 복받쳐\n이것 좀 해라 저것...,사무엘,댄스,용감한 형제
2,10901,10903,Take my hand,That's when I love you\nLove ya love ya love y...,틴탑,댄스,zomay
3,10901,10904,HOTLINE,Boom I like it I like it\nBoom I like it I lik...,승리,댄스,승리
4,10901,10905,Enough,나는 알 수 있어\nWhat makes you sad\n언제든 지칠 때 기대 언제든...,GOT7 (갓세븐),댄스,JAY B
...,...,...,...,...,...,...,...
495,11351,11396,아름다운 밤 (Beautiful Night),따뜻한 바람 불어\n내 맘 속에\n너란 빈틈이 생겨\n꽃이 피네\n뒤척뒤척이다 ba...,Hi-L (하이엘),댄스,PK 헤만
496,11351,11397,GAME (TAG & Ji Beom),눈을 맞춰 My eyes\n이미 시작된 Sign\n그 감각이 그대로 전해져 내게\n...,골든차일드,댄스,MosPick
497,11351,11398,Spell (주문을 걸어),You put a spell on me\n너로 가득 채워\n\n서서히 다가온 그림자...,골든차일드,댄스,진리 (Full8loom)
498,11351,11399,-18,-18 야 우린 만 18세 아냐\nBUT WHEN I’M 18 결국엔 I’LL GE...,트라이비(TRI.BE),댄스,ELLY


In [63]:
songlist = songlist.tail(484)
songlist

,pages,rank,title,lyric,artist,genres,writer
16,10901,10917,처음에 그 마음처럼,*나에겐 푸른 꿈이\n소중한 너와 함께하고픈 꿈이\n너에게 바라는 건\n처음에 그 ...,투니버스,"댄스, 애니메이션/웹툰",박정식
17,10901,10918,PLAYER,날 자극한\n네 심장소리\n애써 숨긴 눈빛이\n왠지 Thrill Me\n이리로 와\...,EVERGLOW (에버글로우),댄스,이스란
18,10901,10919,칵테일 사랑 (Inst.),마음 울적한 날엔 거리를 걸어보고\n향기로운 칵테일에 취해도 보고\n한 편의 시가 ...,여자친구 (GFRIEND),댄스,김선민
19,10901,10920,High Heels,Heels\n눈에 띄게 비춰진\n내 Red Heels\nMy Queen High H...,김완선,댄스,NAASIM
20,10901,10921,거짓말이야,영원한 건 없다던 니 말\n그건 거짓말이야\n아직도 난 영원히 너를\n사랑하고 있으...,임팩트,댄스,HELLO GLOOM
...,...,...,...,...,...,...,...
495,11351,11396,아름다운 밤 (Beautiful Night),따뜻한 바람 불어\n내 맘 속에\n너란 빈틈이 생겨\n꽃이 피네\n뒤척뒤척이다 ba...,Hi-L (하이엘),댄스,PK 헤만
496,11351,11397,GAME (TAG & Ji Beom),눈을 맞춰 My eyes\n이미 시작된 Sign\n그 감각이 그대로 전해져 내게\n...,골든차일드,댄스,MosPick
497,11351,11398,Spell (주문을 걸어),You put a spell on me\n너로 가득 채워\n\n서서히 다가온 그림자...,골든차일드,댄스,진리 (Full8loom)
498,11351,11399,-18,-18 야 우린 만 18세 아냐\nBUT WHEN I’M 18 결국엔 I’LL GE...,트라이비(TRI.BE),댄스,ELLY


In [64]:
songlist.to_pickle('dance_10917-11400.pkl', protocol = 4) # 발라드는 ballad로 저장

In [219]:
songlist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   pages   100 non-null    object
 1   title   100 non-null    object
 2   lyric   100 non-null    object
 3   artist  100 non-null    object
 4   genres  100 non-null    object
 5   writer  100 non-null    object
dtypes: object(6)
memory usage: 4.8+ KB
